In [ ]:
import ee
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=H4-p1tEX3madcGB9_FkYiOgEI1BO1TvnZ8lito9cShw&tc=RqL4jY64ghSur8t4yugbNFy06c_LhcxxdyzIQixZMzk&cc=u2QN-9qWHfiAg9nNqc848Slm1wGqwfkQxiIk7WyjSUo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh7Wc9pubSVee2Y1Wn5DjddNBv2uk2CcBjV_uN91Ty0hKinjLH5Hf9I

Successfully saved authorization token.


In [ ]:
def get_dates(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    dates = []
    while start + timedelta(days=1) <= end:
        dates.append([start.strftime('%Y-%m-%d'), (start + timedelta(days=1)).strftime('%Y-%m-%d')])
        start += timedelta(days=1)
    return dates

In [ ]:
def get_data(start_date, end_date, roi):
    # sentinel 2 dataset
    dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
        .mean()
    # landsat 8 dataset
    dataset2 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .median()
    # dict to keep track of null vals
    is_null = {
        'ndwi': False,
        'mndwi': False,
        'ndci': False,
        'ndti': False,
        'do': False,
        'ndsi': False,
        'ph': False,
        'chl_a': False,
        'ssc': False,
        'wst': False,
    }
    # ndwi
    try:
        ndwi_bands = ['B3', 'B5']
        ndwi = dataset.normalizedDifference(ndwi_bands).rename('ndwi')
        latlon = ee.Image.pixelLonLat().addBands(ndwi)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ndwi = np.array((ee.Array(latlon.get('ndwi')).getInfo()))
    except:
        is_null["ndwi"] = True

    # mndwi
    try:
        band_mndwi = ['B3', 'B6']
        mndwi = dataset.normalizedDifference(band_mndwi).rename('mndwi')
        latlon = ee.Image.pixelLonLat().addBands(mndwi)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_mndwi = np.array((ee.Array(latlon.get('mndwi')).getInfo()))
    except:
        is_null["mndwi"] = True

    # ndci
    try:
        band_ndci = ['B4', 'B3']
        ndci = dataset.normalizedDifference(band_ndci).rename('ndci')
        latlon = ee.Image.pixelLonLat().addBands(ndci)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ndci = np.array((ee.Array(latlon.get('ndci')).getInfo()))
    except:
        is_null["ndci"] = True
    
    # do
    try:
        band_do = ['B8', 'B11']
        dissolvedoxygen = dataset.normalizedDifference(band_do).rename('do')
        latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=geometry,
            scale=8)
        data_do = np.array((ee.Array(latlon.get('do')).getInfo()))
    except: 
        is_null["do"] = True
    # ndti
    try:    
        band_ndti = ['B5', 'B11']
        ndti = dataset.normalizedDifference(band_ndti).rename('ndti')
        latlon = ee.Image.pixelLonLat().addBands(ndti)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ndti = np.array((ee.Array(latlon.get('ndti')).getInfo()))
    except:
        is_null["ndti"] = True
        
    # ndsi
    try:
        band_ndsi = ['B3', 'B11']
        ndsi = dataset.normalizedDifference(band_ndsi).rename('ndsi')
        latlon = ee.Image.pixelLonLat().addBands(ndsi)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ndsi = np.array((ee.Array(latlon.get('ndsi')).getInfo()))
    except:
        is_null["ndsi"] = True
    
    # ph
    try:
        ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(
            dataset.select('B1').divide(dataset.select('B8')))).rename('ph')
        latlon = ee.Image.pixelLonLat().addBands(ph)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ph = np.array((ee.Array(latlon.get('ph')).getInfo()))
    except:
        is_null["ph"] = True

    # chlorophyll a
    try:
        Rrs_red = dataset.select('B3').divide(0.1)
        Rrs_blue = dataset.select('B2').divide(0.1)
        chl_a = Rrs_red.divide(Rrs_blue).pow(2.72).multiply(10.8).rename('chl_a')
        latlon = ee.Image.pixelLonLat().addBands(chl_a)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_chl_a = np.array((ee.Array(latlon.get('chl_a')).getInfo()))
    except:
        is_null["chl_a"] = True

    # suspended sediment conc
    try:
        ssc = ee.Image(0.0113).multiply(ndwi.pow(3)).subtract(ee.Image(0.0135).multiply(
            ndwi.pow(2))).add(ee.Image(0.0075).multiply(ndwi)).add(ee.Image(2.5823)).rename('ssc')
        latlon = ee.Image.pixelLonLat().addBands(ssc)
        latlon = latlon.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=roi,
            scale=8)
        data_ssc = np.array((ee.Array(latlon.get('ssc')).getInfo()))
    except:
        is_null["ssc"] = True

    # WST
    try:
      tir1 = dataset.select('B10')  # 10.8-11.3 µm
      tir2 = dataset.select('B11')  # 11.5-12.5 µm
      #kelvin = tir2.divide(10).add(273.15)
      #wst = kelvin.multiply(1.0158).subtract(
      #    tir1.multiply(0.9543)).subtract(2.7886).rename('wst')
      # latlon = ee.Image.pixelLonLat().addBands(wst)
      #latlon = latlon.reduceRegion(
      #    reducer=ee.Reducer.toList(),
      #    geometry=roi,
      #    scale=8)
      #data_wst = np.array((ee.Array(latlon.get('wst')).getInfo()))
      kelvin = tir2.divide(10).add(273.15)
      lst = kelvin.divide(ee.Image(1).toFloat().divide(tir2.divide(14380).add(1).log()))
      emissivity = ee.Image(0.98)
      atm_corr = lst.multiply(0.99).add(0.11).multiply(emissivity).subtract(2.5)
      wst = atm_corr.subtract(273.15).rename('wst')
      latlon = ee.Image.pixelLonLat().addBands(wst)
      latlon = latlon.reduceRegion(
          reducer=ee.Reducer.toList(),
          geometry=roi,
          scale=8)
      data_wst = np.array((ee.Array(latlon.get('wst')).getInfo()))
    except:
        is_null["wst"] = True
    

    # adding data one by one
    our_data = {}
    
    if is_null["ph"]:
        our_data["ph"] = None
    else:
        our_data["ph"] = data_ph
    
    
    if is_null["wst"]:
        our_data["wst"] = None
    else:
        our_data["wst"] = data_wst

    dataframe = pd.DataFrame(our_data)
    
    return dataframe

In [ ]:
def get_dataframe(start_date, end_date, polygon):
    df_data = pd.DataFrame()
    df_data["NDWI"] = []
    df_data["MNDWI"] = []
    df_data["NDCI"] =[]
    df_data["date"] = []

    dates = get_dates(start_date, end_date)

    for date in dates:
        try:
            df = get_data(date[0], date[1], polygon)
            means = {}
            for column in df.columns:
                try:
                    means[column] = [df[column].mean()]
                except:
                    means[column] = None
            means["date"] = [date[0]]
            mean_data = pd.DataFrame(means)
            df_data = pd.concat([df_data, mean_data])
        except:
            mean_data = {}
            mean_data["date"] = date[0]
            mean_data["NDWI"] = [None]

            mean_data["MNDWI"] = [None]
            mean_data["NDCI"] = [None]
            final_mean = pd.DataFrame(mean_data)
            
            df_data = pd.concat([df_data, final_mean])
    
    return df_data

In [ ]:
geometry =ee.Geometry.Polygon(
        [[[77.39980020154476, 23.209794968246],
          [77.40042247403622, 23.209794968246],
          [77.4008516274786, 23.209755526121455],
          [77.40121640790463, 23.209479430923615],
          [77.40158118833065, 23.209696362912805],
          [77.40143098462582, 23.210011899722915],
          [77.40098037351132, 23.210189388851145],
          [77.40035810101986, 23.210149946843007],
          [77.40115203488827, 23.210603529233456],
          [77.40093745816708, 23.210938784793257],
          [77.40100183118344, 23.21139236450578],
          [77.4014095269537, 23.211767059628617],
          [77.40100183118344, 23.211688176532185],
          [77.40025081265927, 23.211806501159376],
          [77.40014352429867, 23.21147124777687],
          [77.39992894757748, 23.21115571441311],
          [77.39962854016781, 23.210781017575787],
          [77.39945687879086, 23.210386598717097],
          [77.39930667508602, 23.20989357350644]]]);

In [ ]:
start = '2019-12-01'
end = '2022-01-01'

In [ ]:
final_data=get_dataframe(start,end,geometry)